In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk

2024-06-04 02:23:37.130196: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 02:23:37.405514: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with h5py.File('/eos/home-m/mmcohen/ntuples/AOD_EB_ntuples_04-04-2024.h5', 'r') as hf:
    for i in hf.keys():
        print(i+' ' + str(np.array(hf[i]).shape))
    HLT_jets = hf['HLT_jets'][:]
    L1_jFexSR_jets = hf['L1_jFexSR_jets'][:]
    L1_jFexLR_jets = hf['L1_jFexLR_jets'][:]
    HLT_electrons = hf['HLT_electrons'][:]
    LRT_electrons = hf['LRT_electrons'][:]
    L1_egammas = hf['L1_egammas'][:]
    HLT_muons = hf['HLT_muons'][:]
    LRT_muons = hf['LRT_muons'][:]
    L1_muons = hf['L1_muons'][:]
    L1_eFex_taus = hf['L1_eFex_taus'][:]
    L1_jFex_taus = hf['L1_jFex_taus'][:]
    HLT_photons = hf['HLT_photons'][:]
    print(np.array(hf['HLT_MET']).shape)
    HLT_MET = hf['HLT_MET'][:].reshape(-1, 1, 4)  # Broadcasting MET
    L1_MET = hf['L1_MET'][:].reshape(-1, 1, 4)
    #print(L1_MET.shape)
    #print(L1_MET[0,:,:])
    L1_MET_fixed=np.zeros((997315, 1, 3))
    for i,event in enumerate(L1_MET):
        L1_MET_fixed[i,0,0:2]=event[0,0:2]
        L1_MET_fixed[i,0,2]=event[0,3]
    #print(L1_MET_fixed.shape)
    #print(L1_MET_fixed[0,:,:])
    L1_MET=L1_MET_fixed
    pass_L1_unprescaled = hf["pass_L1_unprescaled"][:]
    pass_HLT_unprescaled = hf["pass_HLT_unprescaled"][:]
    EB_weights = hf["EB_weights"][:]


Topo_2A=[L1_muons,L1_eFex_taus,L1_jFex_taus,L1_jFexLR_jets,L1_jFexSR_jets,L1_MET,L1_egammas]
Topo_2A_labels=['L1_muons','L1_eFex_taus','L1_jFex_taus','L1_jFexLR_jets','L1_jFexSR_jets','L1_MET','L1_egammas']
Topo_2B=[L1_egammas,L1_jFexLR_jets,L1_jFexSR_jets,L1_MET,L1_jFex_taus]
Topo_2B_labels=['L1_egammas','L1_jFexLR_jets','L1_jFexSR_jets','L1_MET','L1_jFex_taus']
Topo_3A=[L1_egammas,L1_eFex_taus,L1_jFex_taus,L1_jFexLR_jets,L1_jFexSR_jets,L1_MET]
Topo_3A_labels=['L1_egammas','L1_eFex_taus','L1_jFex_taus','L1_jFexLR_jets','L1_jFexSR_jets','L1_MET']
#Topo_3B=[L1_muons,L1_egamma,L1_eFex_taus]


Topo_2A_concat=Topo_2A[0]
for i in Topo_2A[1:]:
    Topo_2A_concat=np.append(Topo_2A_concat,i,axis=1)
Topo_2A_flat=np.reshape(Topo_2A_concat,(-1,111))

Topo_2B_concat=Topo_2B[0]
for i in Topo_2B[1:]:
    Topo_2B_concat=np.append(Topo_2B_concat,i,axis=1)
Topo_2B_flat=np.reshape(Topo_2B_concat,(-1,75))

Topo_3A_concat=Topo_3A[0]
for i in Topo_3A[1:]:
    Topo_3A_concat=np.append(Topo_3A_concat,i,axis=1)
Topo_3A_flat=np.reshape(Topo_3A_concat,(-1,93))

EB_weights (997315,)
HLT_MET (997315, 4)
HLT_electrons (997315, 3, 4)
HLT_jets (997315, 10, 4)
HLT_muons (997315, 3, 4)
HLT_photons (997315, 3, 4)
L1_MET (997315, 4)
L1_eFex_taus (997315, 6, 3)
L1_egammas (997315, 6, 3)
L1_jFexLR_jets (997315, 6, 3)
L1_jFexSR_jets (997315, 6, 3)
L1_jFex_taus (997315, 6, 3)
L1_muons (997315, 6, 3)
LRT_electrons (997315, 3, 4)
LRT_muons (997315, 3, 4)
lumiblock_bads (997315,)
pass_HLT_unprescaled (997315,)
pass_L1_unprescaled (997315,)
(997315, 4)


In [3]:
Topo_2A_test=Topo_2A_flat[0:450000,:]
Topo_2B_test=Topo_2B_flat[0:450000,:]
Topo_3A_test=Topo_3A_flat[0:450000,:]

Topo_t_weights=EB_weights[0:450000]

In [4]:
 
nans=np.isnan(Topo_2A_test)
a=np.where(nans==True)[0]
for i in a:
    Topo_2A_test[i,92]=0


nans=np.isnan(Topo_2B_test)
b=np.where(nans==True)[0]
for i in b:
    Topo_2B_test[i,56]=0

nans=np.isnan(Topo_3A_test)
c=np.where(nans==True)[0]
for i in c:
    Topo_3A_test[i,92]=0

In [7]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_encoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    x=layers.Dense(h_dim_2, activation='relu')(x)
    z_mean=layers.Dense(latent_dim, name='z_mean')(x)
    z_logvar=layers.Dense(latent_dim, name='z_log_var',kernel_initializer=keras.initializers.Zeros())(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder

def make_encoder_set_weights(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=2001),bias_initializer=keras.initializers.Zeros())(x)
    x=layers.Dense(h_dim_2, activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=2001),bias_initializer=keras.initializers.Zeros())(x)
    z_mean=layers.Dense(latent_dim, name='z_mean',kernel_initializer=keras.initializers.RandomNormal(seed=2001),bias_initializer=keras.initializers.Zeros())(x)
    z_logvar=layers.Dense(latent_dim, name='z_log_var',kernel_initializer=keras.initializers.Zeros(),bias_initializer=keras.initializers.Zeros())(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder

def make_decoder_set_weights(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=2001),bias_initializer=keras.initializers.Zeros())(inputs)
    x=layers.Dense(h_dim_1, activation='relu',kernel_initializer=keras.initializers.RandomNormal(seed=2001),bias_initializer=keras.initializers.Zeros())(x)
    z=layers.Dense(input_dim,kernel_initializer=keras.initializers.RandomNormal(seed=2001),bias_initializer=keras.initializers.Zeros())(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

def make_decoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu')(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    z=layers.Dense(input_dim)(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

class VAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
 

In [ ]:
#TOPO 2A Trainer

beta=0.85
T2A_enc=make_encoder(111,32,16,3)
T2A_dec=make_decoder(111,32,16,3)
T2A=VAE_Model(T2A_enc,T2A_dec)
T2A.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
T2A.compile(optimizer=opt)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = T2A.fit(x=Topo_2A_test, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
T2A.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_2A/Topo_2A_Version_1/',save_format='tf')

In [ ]:
#TOPO 2B Trainer

beta=0.85
T2B_enc=make_encoder(75,32,16,3)
T2B_dec=make_decoder(75,32,16,3)
T2B=VAE_Model(T2B_enc,T2B_dec)
T2B.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
T2B.compile(optimizer=opt)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = T2B.fit(x=Topo_2B_test, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
T2B.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_2B/Topo_2B_Version_1/',save_format='tf')

In [ ]:
#TOPO 3A Trainer

beta=0.85
T3A_enc=make_encoder(93,32,16,3)
T3A_dec=make_decoder(93,32,16,3)
T3A=VAE_Model(T3A_enc,T3A_dec)
T3A.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
T3A.compile(optimizer=opt)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = T3A.fit(x=Topo_3A_test, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
T3A.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_3A/Topo_3A_Version_1/',save_format='tf')

In [ ]:
 #Repeat Trainer Long Train
for i in range(75,100):
    beta=0.85
    T2A_enc=make_encoder(111,32,16,3)
    T2A_dec=make_decoder(111,32,16,3)
    T2A=VAE_Model(T2A_enc,T2A_dec)
    T2A.set_beta(beta)
    opt=keras.optimizers.Adam(learning_rate=0.001)
    T2A.compile(optimizer=opt)

    STOP_PATIENCE = 80
    LR_PATIENCE = 40

    early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

    callbacks = [early_stopping, reduce_lr]

    NUM_EPOCHS = 1000
    BATCH_SIZE = 100

    history = T2A.fit(x=Topo_2A_test, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
    T2A.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Repeat_Testing/2A_long_{}/'.format(i),save_format='tf')

In [8]:
#repeat trainer set bias
for i in range(0,100):
    beta=0.85
    T2A_enc=make_encoder_set_weights(111,32,16,3)
    T2A_dec=make_decoder_set_weights(111,32,16,3)
    T2A=VAE_Model(T2A_enc,T2A_dec)
    T2A.set_beta(beta)
    opt=keras.optimizers.Adam(learning_rate=0.001)
    T2A.compile(optimizer=opt)

    STOP_PATIENCE = 8
    LR_PATIENCE = 4

    early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

    callbacks = [early_stopping, reduce_lr]

    NUM_EPOCHS = 100
    BATCH_SIZE = 100

    history = T2A.fit(x=Topo_2A_test, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
    T2A.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Repeat_Testing/2A_fixed_{}/'.format(i),save_format='tf')

Epoch 1/100
4050/4050 [==============================] - 12s 3ms/step - loss: 357.8849 - reco_loss: 331.1545 - kl_loss: 10.8765 - val_loss: 1171.1201 - val_reco_loss: 91.5802 - val_kl_loss: 1079.5399 - lr: 0.0010
Epoch 2/100
4050/4050 [==============================] - 10s 2ms/step - loss: 106.1861 - reco_loss: 102.5373 - kl_loss: 3.5831 - val_loss: 1088.8995 - val_reco_loss: 74.1691 - val_kl_loss: 1014.7305 - lr: 0.0010
Epoch 3/100
4050/4050 [==============================] - 10s 2ms/step - loss: 85.2246 - reco_loss: 82.0582 - kl_loss: 3.1210 - val_loss: 994.2618 - val_reco_loss: 73.8219 - val_kl_loss: 920.4399 - lr: 0.0010
Epoch 4/100
4050/4050 [==============================] - 10s 2ms/step - loss: 80.3504 - reco_loss: 77.3137 - kl_loss: 3.0085 - val_loss: 948.5459 - val_reco_loss: 72.2854 - val_kl_loss: 876.2606 - lr: 0.0010
Epoch 5/100
4050/4050 [==============================] - 10s 2ms/step - loss: 75.6858 - reco_loss: 72.7922 - kl_loss: 2.9459 - val_loss: 984.3460 - val_reco_lo

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



4050/4050 [==============================] - 10s 2ms/step - loss: 67.0877 - reco_loss: 64.5053 - kl_loss: 2.5804 - val_loss: 858.7838 - val_reco_loss: 64.7262 - val_kl_loss: 794.0576 - lr: 0.0010
Epoch 12/100
4050/4050 [==============================] - 9s 2ms/step - loss: 65.9847 - reco_loss: 63.4138 - kl_loss: 2.5726 - val_loss: 881.4055 - val_reco_loss: 68.8207 - val_kl_loss: 812.5848 - lr: 0.0010
Epoch 13/100
4050/4050 [==============================] - 10s 2ms/step - loss: 66.1518 - reco_loss: 63.5881 - kl_loss: 2.6048 - val_loss: 860.0822 - val_reco_loss: 66.7941 - val_kl_loss: 793.2880 - lr: 0.0010
Epoch 14/100
4050/4050 [==============================] - 9s 2ms/step - loss: 58.6083 - reco_loss: 56.1442 - kl_loss: 2.4781 - val_loss: 854.4917 - val_reco_loss: 65.0564 - val_kl_loss: 789.4353 - lr: 1.0000e-04
Epoch 22/100
4050/4050 [==============================] - 10s 2ms/step - loss: 58.5392 - reco_loss: 56.0612 - kl_loss: 2.4822 - val_loss: 856.4229 - val_reco_loss: 66.7902 - v

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



4050/4050 [==============================] - 10s 2ms/step - loss: 66.7004 - reco_loss: 64.0670 - kl_loss: 2.6252 - val_loss: 937.9115 - val_reco_loss: 55.0836 - val_kl_loss: 882.8279 - lr: 0.0010
Epoch 12/100
4050/4050 [==============================] - 10s 2ms/step - loss: 64.5434 - reco_loss: 61.9559 - kl_loss: 2.5905 - val_loss: 878.3909 - val_reco_loss: 52.8241 - val_kl_loss: 825.5668 - lr: 0.0010
Epoch 13/100
4050/4050 [==============================] - 10s 2ms/step - loss: 64.1291 - reco_loss: 61.5738 - kl_loss: 2.5513 - val_loss: 888.9861 - val_reco_loss: 53.1522 - val_kl_loss: 835.8340 - lr: 0.0010
Epoch 14/100
4050/4050 [==============================] - 10s 2ms/step - loss: 63.5449 - reco_loss: 61.0102 - kl_loss: 2.5440 - val_loss: 853.2865 - val_reco_loss: 55.3451 - val_kl_loss: 797.9414 - lr: 0.0010
Epoch 15/100
4050/4050 [==============================] - 10s 2ms/step - loss: 63.0511 - reco_loss: 60.5565 - kl_loss: 2.5047 - val_loss: 857.2546 - val_reco_loss: 54.3397 - val

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



4050/4050 [==============================] - 10s 2ms/step - loss: 87.3603 - reco_loss: 84.4615 - kl_loss: 2.8193 - val_loss: 903.8062 - val_reco_loss: 68.2337 - val_kl_loss: 835.5725 - lr: 0.0010
Epoch 4/100
4050/4050 [==============================] - 10s 2ms/step - loss: 84.2789 - reco_loss: 81.5273 - kl_loss: 2.7281 - val_loss: 971.4641 - val_reco_loss: 69.1147 - val_kl_loss: 902.3494 - lr: 0.0010
Epoch 5/100
4050/4050 [==============================] - 10s 2ms/step - loss: 77.4314 - reco_loss: 74.7247 - kl_loss: 2.6892 - val_loss: 884.1165 - val_reco_loss: 72.3732 - val_kl_loss: 811.7433 - lr: 0.0010
Epoch 6/100
4050/4050 [==============================] - 10s 2ms/step - loss: 72.9556 - reco_loss: 70.4496 - kl_loss: 2.4815 - val_loss: 803.0878 - val_reco_loss: 73.0639 - val_kl_loss: 730.0239 - lr: 0.0010
Epoch 13/100
4050/4050 [==============================] - 10s 2ms/step - loss: 73.6296 - reco_loss: 71.2031 - kl_loss: 2.4202 - val_loss: 782.8696 - val_reco_loss: 71.1088 - val_kl

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



4050/4050 [==============================] - 10s 2ms/step - loss: 57.8175 - reco_loss: 55.3044 - kl_loss: 2.5170 - val_loss: 838.5792 - val_reco_loss: 56.1567 - val_kl_loss: 782.4225 - lr: 1.0000e-04
Epoch 21/100
4050/4050 [==============================] - 10s 2ms/step - loss: 57.5665 - reco_loss: 55.0418 - kl_loss: 2.5246 - val_loss: 837.7425 - val_reco_loss: 54.8963 - val_kl_loss: 782.8463 - lr: 1.0000e-04
Epoch 22/100
4050/4050 [==============================] - 10s 2ms/step - loss: 80.2053 - reco_loss: 77.1709 - kl_loss: 2.9242 - val_loss: 948.8220 - val_reco_loss: 83.6326 - val_kl_loss: 865.1893 - lr: 0.0010
Epoch 8/100
4050/4050 [==============================] - 10s 2ms/step - loss: 78.4436 - reco_loss: 75.6418 - kl_loss: 2.7508 - val_loss: 893.5424 - val_reco_loss: 72.9908 - val_kl_loss: 820.5516 - lr: 0.0010
Epoch 9/100
4050/4050 [==============================] - 10s 2ms/step - loss: 75.9408 - reco_loss: 73.2198 - kl_loss: 2.7163 - val_loss: 871.7064 - val_reco_loss: 69.9391